In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from pyodesys.symbolic import PartiallySolvedSystem, ScaledSys, symmetricsys, get_logexp
from pycvodes import fpes
from chempy import ReactionSystem, Substance
from chempy.kinetics.ode import get_odesys
sp.init_printing()
%matplotlib inline

In [ ]:
rsys = ReactionSystem.from_string(
    """
    A -> B; 0.04
    B + C -> A + C; 1e4
    2 B -> B + C; 3e7
    """,
    substance_factory=lambda formula: Substance(formula, composition={1: 1}),
    substance_keys='ABC')

In [ ]:
rsys.composition_balance_vectors()

In [ ]:
dep_scaling = 1e9
orisys, extra = get_odesys(rsys, description='original')
scaledsys = ScaledSys.from_other(orisys, dep_scaling=dep_scaling, description='scaled')

In [ ]:
orisys.linear_invariants, orisys.names, orisys.latex_names

In [ ]:
def integrate_systems(systems, **kwargs):
    return [odesys.integrate(
        1e14, {'A': 1, 'B': 0, 'C': 0}, integrator='cvode',
        record_rhs_xvals=True, record_jac_xvals=True, record_order=True, record_fpe=True,
        atol=1e-6, rtol=1e-6, return_on_error=True, **kwargs) for odesys in systems]

def plot_result(odesys, res, ax=None, post_proc=None, colors=('k', 'r', 'g'), xlim=None):
    if ax is None:
        ax = plt.subplot(1, 1, 1)
    vk = 'steps rhs_xvals jac_xvals'.split()  # 'fe_underflow fe_overflow fe_invalid fe_divbyzero'
    res.plot(xscale='log', yscale='log', ax=ax, c=colors,
             info_vlines_kw=dict(vline_keys=vk, post_proc=post_proc))
    lines = ax.get_lines()
    for idx, val in enumerate([1 - 0.04*1e-7, 0.04*1e-7]):
        ax.plot(1e-7, val, marker='o', c=colors[idx])
    for idx, val in enumerate([0.2083340149701255e-7, 0.8333360770334713e-13, 0.9999999791665050]):
        ax.plot(1e11, val, marker='o', c=colors[idx])
    ax.legend(loc='best')
    ax.set_title(odesys.description + (
            ' (%d steps, %d rhs evals., %.5g s CPUt)' % (
                res.info['n_steps'], res.info['nfev'], res.info['time_cpu'])) +
                    ('' if res.info['success'] else ' - failed!'))
    if xlim is not None:
        ax.set_xlim(xlim)
    
def plot_results(systems, results, axes=None, **kwargs):
    if axes is None:
        _fig, axes = plt.subplots(2, 2, figsize=(14, 14))
    for idx, (odesys, res) in enumerate(zip(systems, results)):
        plot_result(odesys, res, ax=axes.flat[idx], **kwargs)

In [ ]:
plot_results([orisys, scaledsys], integrate_systems([orisys, scaledsys], nsteps=5000, first_step=1e-23),
             axes=plt.subplots(1, 2, figsize=(14, 7))[1], xlim=(1e-23, 1e14))

In [ ]:
psys = [PartiallySolvedSystem.from_linear_invariants(scaledsys, [k], description=k) for k in 'ABC']
linsystems = [scaledsys] + psys
[cs.dep for cs in psys]

In [ ]:
linresults = integrate_systems(linsystems, nsteps=5000)

In [ ]:
plot_results(linsystems, linresults, xlim=(1e-9, 1e14))

In [ ]:
indep0 = 1e-26
LogLogSys = symmetricsys(get_logexp(1, 1e-20, b2=0), get_logexp(1, indep0, b2=0), exprs_process_cb=lambda exprs: [
                         sp.powsimp(expr.expand(), force=True) for expr in exprs])
logsystems = [LogLogSys.from_other(ls) for ls in linsystems]

In [ ]:
logresults = integrate_systems(logsystems, nsteps=15000)

In [ ]:
lnds = np.log(indep0)
plot_results(logsystems, logresults, post_proc=lambda x: np.abs(np.exp(x) - indep0), xlim=[indep0*10, 1e14])

In [ ]:
logsystems[0].exprs